## MLFlow Tracking Server

In order to start the MLflow tracking server,  go to the folder via cmd or powershell, and type:  mlflow ui

In [1]:
import mlflow
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from mlflow.models import infer_signature



In [2]:
# Load Iris dataset

X, y = datasets.load_iris(return_X_y=True)

# Split the dat ainto training and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [3]:
#Define the model hiperparameters

params = {"penalty":"l2","solver": "lbfgs", "max_iter": 1000, "multi_class": "auto", "random_state": 8888}

# Train the model

lr=LogisticRegression(**params)
lr.fit(X_train, y_train)



c:\Users\jmari\OneDrive\CCT\MsC_Data_Analytics\MLFlow_Project\mlflow\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888)

In [4]:
# Predict on the test set
y_pred = lr.predict(X_test)
y_pred

array([0, 0, 1, 2, 1, 0, 1, 0, 2, 2, 1, 2, 0, 0, 1, 1, 0, 1, 1, 2, 1, 2,
       2, 2, 0, 2, 0, 0, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 2, 0, 1, 2, 0, 2,
       0])

In [5]:
# Acuracy score

accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9555555555555556

In [6]:
# MLFlow tracking
mlflow.set_tracking_uri("http://localhost:5000")

#Set experiment
mlflow.set_experiment('Track LR Simple Model')

# Start a run
with mlflow.start_run():
    # log hyperparameters
    mlflow.log_params(params)
    # log model accuracy
    mlflow.log_metric("accuracy", accuracy)
    # set tag for experiment
    mlflow.set_tag("Training Info", "Basic LR Model on Iris Dataset")
    # infer signature of the model
    signature = infer_signature(X_train, lr.predict(X_train))
    # log model
    model_info = mlflow.sklearn.log_model(
        sk_model = lr,
         artifact_path = "iris_model",
         signature = signature,
         input_example = X_train,
         registered_model_name = "LR iris_classifier"
    )


Registered model 'LR iris_classifier' already exists. Creating a new version of this model...
2024/10/24 18:49:22 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LR iris_classifier, version 6
Created version '6' of model 'LR iris_classifier'.
2024/10/24 18:49:27 INFO mlflow.tracking._tracking_service.client: 🏃 View run trusting-penguin-152 at: http://localhost:5000/#/experiments/211716385583910669/runs/8dc5d125710b43c79d67d783b1ea856c.
2024/10/24 18:49:27 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/211716385583910669.


### Changing parameters and retraining the model

In [7]:
# Define the model hyperparameters
params = {"solver": "newton-cg", "max_iter": 1000, "multi_class": "auto", "random_state": 1000}

##train the model

lr=LogisticRegression(**params)
lr.fit(X_train,y_train)


c:\Users\jmari\OneDrive\CCT\MsC_Data_Analytics\MLFlow_Project\mlflow\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=1000,
                   solver='newton-cg')

In [8]:
y_pred=lr.predict(X_test)
y_pred

array([0, 0, 1, 2, 1, 0, 1, 0, 2, 2, 1, 2, 0, 0, 1, 1, 0, 1, 1, 2, 1, 2,
       2, 2, 0, 2, 0, 0, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 2, 0, 1, 2, 0, 2,
       0])

In [9]:
accuracy=accuracy_score(y_test,y_pred)
print(accuracy)

0.9555555555555556


In [10]:
# MLFlow tracking
mlflow.set_tracking_uri("http://localhost:5000")

#Set experiment
mlflow.set_experiment('Track LR Model')

# Start a run
with mlflow.start_run():
    # log hyperparameters
    mlflow.log_params(params)
    # log model accuracy
    mlflow.log_metric("accuracy", accuracy)
    # set tag for experiment
    mlflow.set_tag("Training Info", "Basic LR Model on Iris Dataset")
    # infer signature of the model
    signature = infer_signature(X_train, lr.predict(X_train))
    # log model
    model_info = mlflow.sklearn.log_model(
        sk_model = lr,
         artifact_path = "iris_model",
         signature = signature,
         input_example = X_train,
         registered_model_name = "LR iris_classifier "
    )


Registered model 'LR iris_classifier ' already exists. Creating a new version of this model...
2024/10/24 18:50:32 INFO mlflow.tracking._tracking_service.client: 🏃 View run popular-mule-801 at: http://localhost:5000/#/experiments/990330760718679012/runs/6d33c17be89646bd80764cc6c5eef210.
2024/10/24 18:50:32 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/990330760718679012.


MlflowException: API request to http://localhost:5000/api/2.0/mlflow/model-versions/create failed with exception HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/2.0/mlflow/model-versions/create (Caused by ResponseError('too many 500 error responses'))

# Inferencing and VAlidating the Model

In [ ]:
from mlflow.models import validate_serving_input

model_uri = 'runs:/363c6234df5b4901927f2a817bbe7a7f/iris_model'

# The model is logged with an input example. MLflow converts
# it into the serving payload format for the deployed model endpoint,
# and saves it to 'serving_input_payload.json'
serving_payload = """{
  "inputs": [
    [
      6.6,
      2.9,
      4.6,
      1.3
    ],
    [
      4.7,
      3.2,
      1.3,
      0.2
    ],
    [
      5.1,
      3.8,
      1.6,
      0.2
    ],
    [
      7.7,
      2.8,
      6.7,
      2.0
    ],
    [
      6.5,
      3.0,
      5.2,
      2.0
    ],
    [
      5.7,
      3.0,
      4.2,
      1.2
    ],
    [
      5.5,
      2.6,
      4.4,
      1.2
    ],
    [
      6.4,
      2.9,
      4.3,
      1.3
    ],
    [
      6.0,
      2.2,
      4.0,
      1.0
    ],
    [
      7.6,
      3.0,
      6.6,
      2.1
    ],
    [
      5.5,
      2.4,
      3.7,
      1.0
    ],
    [
      6.3,
      2.5,
      4.9,
      1.5
    ],
    [
      5.2,
      4.1,
      1.5,
      0.1
    ],
    [
      5.5,
      3.5,
      1.3,
      0.2
    ],
    [
      6.9,
      3.2,
      5.7,
      2.3
    ],
    [
      6.9,
      3.1,
      5.4,
      2.1
    ],
    [
      6.7,
      3.3,
      5.7,
      2.1
    ],
    [
      5.2,
      3.4,
      1.4,
      0.2
    ],
    [
      5.5,
      2.4,
      3.8,
      1.1
    ],
    [
      4.9,
      2.4,
      3.3,
      1.0
    ],
    [
      7.3,
      2.9,
      6.3,
      1.8
    ],
    [
      5.4,
      3.4,
      1.5,
      0.4
    ],
    [
      4.9,
      3.1,
      1.5,
      0.1
    ],
    [
      5.7,
      4.4,
      1.5,
      0.4
    ],
    [
      5.7,
      2.8,
      4.1,
      1.3
    ],
    [
      5.0,
      3.6,
      1.4,
      0.2
    ],
    [
      6.4,
      2.8,
      5.6,
      2.1
    ],
    [
      6.0,
      2.2,
      5.0,
      1.5
    ],
    [
      5.0,
      3.0,
      1.6,
      0.2
    ],
    [
      6.5,
      2.8,
      4.6,
      1.5
    ],
    [
      6.1,
      2.6,
      5.6,
      1.4
    ],
    [
      5.0,
      3.5,
      1.6,
      0.6
    ],
    [
      4.5,
      2.3,
      1.3,
      0.3
    ],
    [
      5.0,
      3.4,
      1.5,
      0.2
    ],
    [
      7.9,
      3.8,
      6.4,
      2.0
    ],
    [
      4.9,
      3.0,
      1.4,
      0.2
    ],
    [
      7.7,
      3.0,
      6.1,
      2.3
    ],
    [
      5.0,
      2.0,
      3.5,
      1.0
    ],
    [
      6.6,
      3.0,
      4.4,
      1.4
    ],
    [
      5.1,
      3.4,
      1.5,
      0.2
    ],
    [
      6.3,
      2.8,
      5.1,
      1.5
    ],
    [
      5.0,
      3.3,
      1.4,
      0.2
    ],
    [
      6.7,
      3.1,
      4.7,
      1.5
    ],
    [
      5.8,
      2.6,
      4.0,
      1.2
    ],
    [
      5.5,
      2.5,
      4.0,
      1.3
    ],
    [
      5.0,
      3.5,
      1.3,
      0.3
    ],
    [
      6.3,
      2.9,
      5.6,
      1.8
    ],
    [
      6.8,
      3.2,
      5.9,
      2.3
    ],
    [
      5.4,
      3.4,
      1.7,
      0.2
    ],
    [
      5.8,
      4.0,
      1.2,
      0.2
    ],
    [
      5.2,
      2.7,
      3.9,
      1.4
    ],
    [
      6.0,
      3.0,
      4.8,
      1.8
    ],
    [
      5.6,
      2.7,
      4.2,
      1.3
    ],
    [
      6.1,
      2.8,
      4.0,
      1.3
    ],
    [
      6.2,
      2.9,
      4.3,
      1.3
    ],
    [
      5.0,
      2.3,
      3.3,
      1.0
    ],
    [
      5.6,
      2.8,
      4.9,
      2.0
    ],
    [
      5.1,
      3.7,
      1.5,
      0.4
    ],
    [
      7.7,
      3.8,
      6.7,
      2.2
    ],
    [
      6.3,
      2.5,
      5.0,
      1.9
    ],
    [
      6.2,
      3.4,
      5.4,
      2.3
    ],
    [
      5.4,
      3.0,
      4.5,
      1.5
    ],
    [
      5.7,
      2.8,
      4.5,
      1.3
    ],
    [
      4.7,
      3.2,
      1.6,
      0.2
    ],
    [
      6.1,
      3.0,
      4.9,
      1.8
    ],
    [
      5.1,
      3.8,
      1.9,
      0.4
    ],
    [
      6.0,
      3.4,
      4.5,
      1.6
    ],
    [
      5.4,
      3.9,
      1.3,
      0.4
    ],
    [
      5.7,
      2.5,
      5.0,
      2.0
    ],
    [
      5.5,
      4.2,
      1.4,
      0.2
    ],
    [
      6.4,
      3.2,
      5.3,
      2.3
    ],
    [
      5.6,
      3.0,
      4.1,
      1.3
    ],
    [
      6.9,
      3.1,
      4.9,
      1.5
    ],
    [
      6.7,
      3.0,
      5.2,
      2.3
    ],
    [
      5.6,
      3.0,
      4.5,
      1.5
    ],
    [
      6.4,
      3.1,
      5.5,
      1.8
    ],
    [
      4.9,
      2.5,
      4.5,
      1.7
    ],
    [
      4.6,
      3.1,
      1.5,
      0.2
    ],
    [
      4.9,
      3.1,
      1.5,
      0.2
    ],
    [
      4.3,
      3.0,
      1.1,
      0.1
    ],
    [
      6.7,
      3.0,
      5.0,
      1.7
    ],
    [
      6.0,
      2.7,
      5.1,
      1.6
    ],
    [
      6.4,
      3.2,
      4.5,
      1.5
    ],
    [
      5.1,
      3.5,
      1.4,
      0.3
    ],
    [
      6.5,
      3.0,
      5.5,
      1.8
    ],
    [
      5.9,
      3.0,
      4.2,
      1.5
    ],
    [
      4.9,
      3.6,
      1.4,
      0.1
    ],
    [
      4.6,
      3.4,
      1.4,
      0.3
    ],
    [
      7.2,
      3.0,
      5.8,
      1.6
    ],
    [
      6.7,
      3.1,
      4.4,
      1.4
    ],
    [
      6.1,
      3.0,
      4.6,
      1.4
    ],
    [
      7.1,
      3.0,
      5.9,
      2.1
    ],
    [
      6.9,
      3.1,
      5.1,
      2.3
    ],
    [
      7.2,
      3.2,
      6.0,
      1.8
    ],
    [
      6.7,
      3.3,
      5.7,
      2.5
    ],
    [
      7.0,
      3.2,
      4.7,
      1.4
    ],
    [
      5.1,
      3.8,
      1.5,
      0.3
    ],
    [
      5.4,
      3.7,
      1.5,
      0.2
    ],
    [
      4.8,
      3.1,
      1.6,
      0.2
    ],
    [
      5.8,
      2.8,
      5.1,
      2.4
    ],
    [
      6.4,
      2.8,
      5.6,
      2.2
    ],
    [
      6.1,
      2.8,
      4.7,
      1.2
    ],
    [
      6.2,
      2.8,
      4.8,
      1.8
    ],
    [
      4.4,
      3.0,
      1.3,
      0.2
    ],
    [
      5.9,
      3.0,
      5.1,
      1.8
    ]
  ]
}"""

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

# Inferencing the model as a generic  function


In [ ]:
model_uri = 'runs:/363c6234df5b4901927f2a817bbe7a7f/iris_model'

loaded_model = mlflow.pyfunc.load_model(model_uri)

predictions = loaded_model.predict(X_test)

iris_features_name = datasets.load_iris().feature_names
result = pd.DataFrame(X_test, columns=iris_features_name)
result["actual_class"] = y_test
result["predicted_class"] = predictions
result.head()

# Inferencing the model from regisered model

In [ ]:
import mlflow.sklearn
model_name = 'LR iris_classifier'
model_version = 'latest'


model_url = f"models:/{model_name}/{model_version}"

model=mlflow.sklearn.load_model(model_url)

model

In [ ]:
model.predict(X_test)